In [1]:
!pip install --upgrade paho-mqtt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 KB 776.7 kB/s eta 0:00:00a 0:00:01


In [2]:
!pip install pytz

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import paho.mqtt.client as mqtt
import json
from kafka import KafkaProducer

# Configuration du producteur Kafka
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],  # Remplacez par l'adresse de votre broker Kafka
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Callback pour la connexion
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    # Souscription au topic
    client.subscribe("/hfp/v2/journey/#")  # Remplacez par le topic souhaité

# Callback pour la réception d'un message
def on_message(client, userdata, msg):
    try:
        # Décoder le message reçu en UTF-8
        message_data = json.loads(msg.payload.decode('utf-8'))
        
        # Afficher le message reçu
        print(f"Message reçu sur le topic {msg.topic} :")
        print(json.dumps(message_data, indent=4))  # Affichage formaté pour une lecture facile

        # Vous pouvez envoyer directement à Kafka
        # Le message sera envoyé au topic Kafka 'vip'
        producer.send('vip', value=message_data)
        print(f"Message envoyé à Kafka : {json.dumps(message_data)}")
        
    except json.JSONDecodeError:
        print("Erreur de décodage du JSON")
        print(f"Payload reçu : {msg.payload.decode('utf-8')}")
    except TypeError as e:
        print(f"Erreur de type : {e}")
        print(f"Message problématique : {msg.payload.decode('utf-8')}")

# Création du client MQTT
client = mqtt.Client()

# Assignation des callbacks
client.on_connect = on_connect
client.on_message = on_message

# Configuration de la connexion sécurisée (si nécessaire)
client.tls_set()  # Active TLS pour la connexion sécurisée si le broker le nécessite

# Connexion au broker MQTT
client.connect("mqtt.hsl.fi", 8883, 60)  # Remplacez l'adresse et le port selon votre broker

# Lancer la boucle pour écouter les messages
client.loop_start()

# Attendre un certain temps pour récupérer les messages
import time
time.sleep(2)  # Attendre 60 secondes pour recevoir des messages

# Arrêter la boucle après le délai
client.loop_stop()


/tmp/ipykernel_4713/4042499085.py:40: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected with result code 0
Message reçu sur le topic /hfp/v2/journey/ongoing/doo/bus/0018/00697/2224/2/Leppävaara as./17:00/2611267/5/60;24/26/07/96 :
{
    "DOO": {
        "desi": "224",
        "dir": "2",
        "oper": 6,
        "veh": 697,
        "tst": "2024-12-26T15:01:03.755Z",
        "tsi": 1735225263,
        "spd": 0.15,
        "hdg": 247,
        "lat": 60.209196,
        "long": 24.676668,
        "acc": -0.21,
        "dl": 57,
        "odo": 547,
        "drst": 1,
        "oday": "2024-12-26",
        "jrn": 1329,
        "line": 911,
        "start": "17:00",
        "loc": "GPS",
        "stop": 2611267,
        "route": "2224",
        "occu": 0,
        "ttarr": "2024-12-26T15:02:00.000Z",
        "ttdep": "2024-12-26T15:02:00.000Z"
    }
}
Message envoyé à Kafka : {"DOO": {"desi": "224", "dir": "2", "oper": 6, "veh": 697, "tst": "2024-12-26T15:01:03.755Z", "tsi": 1735225263, "spd": 0.15, "hdg": 247, "lat": 60.209196, "long": 24.676668, "acc": -0.21, "dl": 5

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>